In [1]:
# https://www.chosun.com/culture-life/culture_general/?page=2 식

section_dict={"economy/economy_general":"경제","politics/politics_general":"정치","national/national_general":"사회","international/international_general":"국제","sports/sports_general":"스포츠","entertainments/enter_general":"연예","culture-life/culture_general":"문화"}

In [4]:
sections = list(section_dict.keys())

In [5]:
sections

['economy/economy_general',
 'politics/politics_general',
 'national/national_general',
 'international/international_general',
 'sports/sports_general',
 'entertainments/enter_general',
 'culture-life/culture_general']

In [80]:
import requests
from bs4 import BeautifulSoup
import time
import random
import datetime
import numpy as np
import pandas as pd

In [81]:
def filesave(dataframe,name):
    dataframe.to_csv(name,sep="\t", encoding="utf-8",index=False)
def fileread(name):
    return pd.read_csv(name,sep="\t",encoding="utf-8")

In [130]:
def chosun_make_link_list(section_name,pageidx):
    url = "https://www.chosun.com/{}/?page={}".format(section_name,pageidx)
    
    driver.implicitly_wait(1)
    driver.get(url)
    html = driver.page_source
    html = BeautifulSoup(html, 'html.parser')
    news_list=html.findAll("a",attrs={'class': 'text__link'})
    news_link_list=[]
    cnt=0
    for i in news_list:
        news_link_list.append(str(i.attrs['href']))
        if cnt>=18:
            break
        cnt+=1
    return news_link_list

In [131]:
def chosun_select_section_auto(sectionlist,startpage,stoppage):
    autolinklist=[]
    for section in sectionlist:
        for i in range(startpage,stoppage):
            temp=chosun_make_link_list(section,i)
            if not temp:
                continue
            autolinklist+=temp
        time.sleep(abs((np.random.randn())*0.1))

    return autolinklist

In [143]:
def chosun_link_to_df(link_list):
    root_url="https://www.chosun.com"
    data=[]
    

    for link in link_list:
        url = root_url+link
        driver.implicitly_wait(0.5)
        driver.get(url)
        html = driver.page_source

        pagehtml = BeautifulSoup(html, 'html.parser')
        
        
        
        pagetitle=pagehtml.h1
        pagecontent=pagehtml.find("section",attrs={'class': 'article-body'})
        pagedate=pagehtml.find("div",attrs={'class': 'article-dateline'})
        
        temp=url.split(".chosun.com/")
        temp=temp[-1].split("/2")
        pagesection = temp[0]
        
        if temp[0] in sections:
            pagesection = section_dict[temp[0]]
        
        if pagetitle and pagecontent and pagedate and pagesection:
            pagetitle=pagetitle.text
            pagecontent=pagecontent.text
            pagedate=pagedate.text.replace("입력","")
            
        data.append([pagetitle,pagecontent,pagedate,pagesection,url])
        
        time.sleep(abs((np.random.randn())*0.05))
        
    return pd.DataFrame(data, columns=['title','content','date','section','url'])

In [133]:
#startdate를 읽어온 csv 파일에서 가장 최신 날짜로 자동으로 잡도록...하고 싶었지만 생각해보니 애초에 날짜 저장을 안 했다
def auto_save(startpage,pages=3):
    url_df = fileread("chosun_news_url.csv")
    c=chosun_select_section_auto(sections,startpage,startpage+pages)
    url_df_temp = pd.DataFrame(c,columns=['news_url'])
    url_df = pd.concat([url_df,url_df_temp])
    filesave(url_df,"chosun_news_url.csv")
    print(startpage+pages,"까지 저장")

In [103]:
driver = webdriver.Chrome('C:/Users/user/cnu_crawling/chromedriver_win32/chromedriver')
url_df=pd.DataFrame(columns=['news_url'])

<ipython-input-103-4957f6b60c99>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/user/cnu_crawling/chromedriver_win32/chromedriver')


In [104]:
filesave(url_df,"chosun_news_url.csv")

In [107]:
count=181

for i in range(60):
    auto_save(count)
    count+=3

184 까지 저장
187 까지 저장
190 까지 저장
193 까지 저장
196 까지 저장
199 까지 저장
202 까지 저장
205 까지 저장
208 까지 저장


KeyboardInterrupt: 

In [108]:
test=fileread("chosun_news_url.csv")

In [109]:
test

,news_url
0,/economy/economy_general/2022/10/25/L6LM4QPAZF...
1,/economy/economy_general/2022/10/25/HH3S6YI6CN...
2,/economy/economy_general/2022/10/25/IIDTJWYION...
3,/economy/economy_general/2022/10/25/RREKVAFDBJ...
4,/economy/economy_general/2022/10/25/HGFE3YHQYZ...
...,...
24825,/culture-life/culture_general/2021/04/05/CSH6G...
24826,/culture-life/2021/04/05/PHANEHPXYVAQJOZUK5KBO...
24827,/culture-life/culture_general/2021/04/04/OC7Q4...
24828,/culture-life/2021/04/03/BFLHKQCUGNF7PKUQRBSU7...


In [124]:
test=fileread("chosun_news_url.csv")
splitedlist=[test[i*10:i*10+10] for i in range(0,(len(test)//10)+1)]
print("length of for : ",len(splitedlist))

length of for :  2484


In [134]:
chosun_df = fileread("chosun_dataframe.csv")

In [ ]:
driver = webdriver.Chrome('C:/Users/user/cnu_crawling/chromedriver_win32/chromedriver')
count=36
for _ in range(5):
    try:
        for linklist in splitedlist[count:]:
            start = time.time()
            print("start")
            chosun_df_temp = chosun_link_to_df(list(linklist['news_url']))
            chosun_df = pd.concat([chosun_df,chosun_df_temp])
            filesave(chosun_df,"chosun_dataframe.csv")
            print("saved")
            count+=1

            print("소요시간 :", time.time() - start)
            print(count*10,"번째까지 저장")

            time.sleep((random.random()*0.2))
    except:
        print(count,"에서 에러 발생")
        time.sleep(3)

<ipython-input-144-8d7ae6259a02>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/user/cnu_crawling/chromedriver_win32/chromedriver')


In [142]:
driver = webdriver.Chrome('C:/Users/user/cnu_crawling/chromedriver_win32/chromedriver')
count=36

for linklist in splitedlist[count:]:
    start = time.time()
    print("start")
    chosun_df_temp = chosun_link_to_df(list(linklist['news_url']))
    chosun_df = pd.concat([chosun_df,chosun_df_temp])
    filesave(chosun_df,"chosun_dataframe.csv")
    print("saved")
    count+=1

    print("소요시간 :", time.time() - start)
    print(count*10,"번째까지 저장")

    time.sleep((random.random()*0.2))

<ipython-input-142-a60af6c08eb1>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/user/cnu_crawling/chromedriver_win32/chromedriver')


start


UnboundLocalError: local variable 'pagesection' referenced before assignment

In [135]:
chosun_df

,title,content,date,section,url
0,"신한금융, 3분기 순익 1.6조… 역대 최대 실적",조용병 신한금융지주 회장이 고객 중심 경영을 강조하고 있다.신한금융이 올해 3분기(...,2022.10.25 13:52,경제,https://www.chosun.com/economy/economy_general...
1,자금시장 경색에… 한은 ‘베이비스텝’ 전망 나와,‘레고랜드 사태’가 뇌관이 된 자금 시장 경색이 쉽게 풀릴 기미를 보이지 않으면서 ...,2022.10.25 04:16,경제,https://www.chosun.com/economy/economy_general...
2,일본은행 두번째 복면개입? 엔·달러 환율 149엔대에서 갑자기 4엔 하락,지난 20일 스즈키 슌이치 일본 재무상이 취재진과 만나는 모습. /로이터 연합뉴스달...,2022.10.25 03:00,경제,https://www.chosun.com/economy/economy_general...
3,매달 10만원씩 6년 부었는데 이자 12만원? 낮아도 너무 낮은 금리에 ‘저축보험’...,은행 수신금리가 나날이 오르자 오래 전 저축보험을 가입했다가 중도해지에 나서는 사람...,2022.10.25 03:00,경제,https://www.chosun.com/economy/economy_general...
4,"작년 종부세 수정 요구 1481건, 1년새 80% 폭증… 절반이 받아들여졌다",지난해 납세자들이 종합부동산세(종부세)를 과도하게 냈다며 수정을 요구한 경정 청구가...,2022.10.25 03:00,경제,https://www.chosun.com/economy/economy_general...
...,...,...,...,...,...
75,환노위 “제빵공장 잇단 사고...정말 참담한 심정”,강동석 SPL 대표이사가 24일 국회에서 열린 환경노동위원회 고용노동부 등에 관한 ...,2022.10.24 22:14,정치,https://www.chosun.com/politics/politics_gener...
76,10대부터 50대 노병까지... 6·25 참전 美영웅들 책 나온다,6·25전쟁에 참전한 미국 영웅들의 이야기가 책으로 나온다. 1953년 6월 백병전...,2022.10.24 20:20,정치,https://www.chosun.com/politics/politics_gener...
77,“눈물에 약한 국민 심성 악용”...이재명 눈물에 과거 발언 소환,이재명 더불어민주당 대표가 검찰이 민주연구원을 압수수색한 24일 오전 민주당 중앙당...,2022.10.24 17:17,정치,https://www.chosun.com/politics/politics_gener...
78,"김의겸 “尹과 청담동 술집 갔죠?”… 한동훈 “다 걸고 안 갔다, 의원님 뭐 걸겠냐”",한동훈 법무부 장관이 국정감사장에서 김의겸 더불어민주당 의원의 의혹 제기에 강한 분...,2022.10.24 17:09,정치,https://www.chosun.com/politics/politics_gener...
